<a href="https://colab.research.google.com/github/lahirumadushanfernando/ContractUtilizationRecommmender/blob/main/ContractUtilizationHackathonV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
# importing the libraries
import pandas as pd
from difflib import SequenceMatcher
import numpy as np
import re
from fuzzywuzzy import process, fuzz
from difflib import SequenceMatcher
from google.colab import files
import warnings
warnings.filterwarnings("ignore")
%autosave 120

Autosaving every 120 seconds


In [3]:
# definition of source path
path = 'https://raw.githubusercontent.com/lahirumadushanfernando/ContractUtilizationRecommmender/main/AllContracts.csv'
sheet_name = 'AllContracts'

In [4]:
# loading data into a data frame
data = pd.read_csv(path, encoding = 'unicode_escape')
# renaming the columns
# data.rename(columns={'Deletion indicator': 'DeletionIndicator', 'Long Text' : 'LongText'}, inplace=True)
data.shape
# data.head(2)

(30000, 24)

In [5]:
# loading the deleted lines into a data frame
deleted_lines = data.query("DELETION_INDICATOR in ('L','S')")
deleted_lines.shape

(6182, 24)

In [6]:
# removing the deleted lines from original data
active_data = pd.concat([data, deleted_lines]).drop_duplicates(keep=False)
active_data.shape

(23818, 24)

In [7]:
# loading the CCTR order types into a data frame
cctr_lines = active_data.query("ORDER_TYPE in ('CCTR')")
cctr_lines.shape

(6610, 24)

In [8]:
# removing the CCTR lines from active data
active_data = pd.concat([active_data, cctr_lines]).drop_duplicates(keep=False)
active_data.shape

(17208, 24)

In [9]:
# Examining the data type of the column
# active_data.VALIDITY_START_DATE.dtype

# Getting the year and month part of the VALIDITY_START_DATE
active_data['YEAR'] = pd.DatetimeIndex(active_data['VALIDITY_START_DATE']).year
active_data['MONTH'] = pd.DatetimeIndex(active_data['VALIDITY_START_DATE']).month

In [10]:
active_data['LONG_TEXT_DES'] = active_data['LONG_TEXT'].str.split('-').str[0] 
active_data['LONG_TEXT_DES'] = active_data['LONG_TEXT_DES'].str.replace(' ', '')
active_data['LONG_TEXT_DES'] = active_data['LONG_TEXT_DES'].str.upper()

In [11]:
# Combine Year, Month, OrderType
active_data['CLUSTER_NAME'] = active_data['LONG_TEXT_DES'] + active_data['ORDER_TYPE'] +  active_data['YEAR'].astype(str) + active_data['MONTH'].astype(str) 
# active_data.head(2)

In [12]:
# test_df = active_data[active_data['LONG_TEXT'] == 4100000437] 
# Getting a copy of dataframe from original with only two columns
cluster_df = active_data[['CLUSTER_NAME','VENDOR_OR_CREDITOR_CODE']].copy()
cluster_df = cluster_df.dropna(how='any',axis=0)
# cluster_df

In [13]:
# # Getting a sub data frame starting names with "TEA"
# cluster_df_tea = cluster_df.loc[test_df['CLUSTER_NAME'].str.startswith('TEA')].copy()
# cluster_df_tea

In [14]:
# Checking out qunique clusters within data frame
unique_clusters = cluster_df.CLUSTER_NAME.unique()
unique_clusters_df = pd.DataFrame(unique_clusters)
unique_clusters_df.columns =['UNIQUE_CLUSTER_NAME']
print(unique_clusters_df)

                             UNIQUE_CLUSTER_NAME
0       SUPPLY&DELIVERYOFAIRCRAFTWARNINGLMK20119
1       SUPPLYOFSOUNDPROOFINGOFOPENTYPEGEMK20111
2     SUPPLY,DELIVERY,OFSPARESFORRECTIFIEMK20112
3                         COMMERCIALORDERMK20114
4                                       PMK20114
...                                          ...
1434             PAYMENTOFBBCEARTHCHANNELWK20227
1435   REMOVALSOFEQUIPMENTRELATEDTODTVANDWK20227
1436     FIREWALLSECURITYSOLUTIONSERVICESWK20227
1437   REMOVALSOFEQUIPMENTRELATEDTODTVANDWK20228
1438    FULLLEVELREPAIROFLTE,DTV,DEDEVICEWK20224

[1439 rows x 1 columns]


In [15]:
# # Get the total count of rows per unique cluster, not passing the distinct row count per cluster
# test_df_vendor_count = test_df_tea.groupby('CLUSTER_NAME')['VENDOR_OR_CREDITOR_CODE'].count()
# test_df_vendor_count

In [16]:
# Getting the number of unique vendors assgined per unique cluster
cluster_df_vendor_count_arry = cluster_df.groupby('CLUSTER_NAME')['VENDOR_OR_CREDITOR_CODE'].nunique()
cluster_df_vendor_count = pd.DataFrame(cluster_df_vendor_count_arry)
cluster_df_vendor_count.rename(columns = {'VENDOR_OR_CREDITOR_CODE':'VENDOR_COUNT'}, inplace = True)
# cluster_df_vendor_count

In [17]:
# Removing the clusters where vendor count is less than 1
contracts_cluster_df = cluster_df_vendor_count[cluster_df_vendor_count['VENDOR_COUNT'] > 1]

# Indexing the clusters starting from 0
contracts_cluster_df['CLUSTER_NUM'] = contracts_cluster_df.reset_index().index
contracts_cluster_df

,VENDOR_COUNT,CLUSTER_NUM
CLUSTER_NAME,,
10016547WK20117,3,0
ACBREAKDOWNMAINTENANCEATDIALOGSITESWK20148,9,1
"ACCESSORIES,BATTERIESANDADAPTORSRATEMK20146",2,2
"ACCESSORIES,BATTERIESANDADAPTORSWK20214",2,3
ADMINPRINTINGANDMERCHANDISINGRATECAMK20156,7,4
...,...,...
TOWERANDCABINMAINTENANCESWK20113,5,217
TOWERANDCABINMAINTENANCEWK201910,10,218
TRANSPORTATIONCONTRACTWK201910,3,219


In [18]:
# Merging the orginal data set with the clustered contracts data set
final_data_arry = pd.merge(active_data, contracts_cluster_df, on='CLUSTER_NAME',how='left')
final_data = pd.DataFrame(final_data_arry)
final_data

,CONTRACT_ID,PURCH_DOC_ITEM_NO,ORDER_TYPE,VENDOR_OR_CREDITOR_CODE,SHORT_TEXT,CURRENCY_KEY,TARGET_VALUE_HEADER_AREA_PER_DISTRIBUTION,RELEASED_VALUE,TOTAL_OPEN_VALUE,VALIDITY_START_DATE,...,PURCH_GROUP,LONG_TEXT,SALES_PERSON,DELETION_INDICATOR,YEAR,MONTH,LONG_TEXT_DES,CLUSTER_NAME,VENDOR_COUNT,CLUSTER_NUM
0,4100000000,10,MK,100029,MODULE AC 650 352 -&- Power & utility,LKR,0.00,0.00,0.00,3/30/2011,...,004,NaN,NaN,NaN,2011,3,NaN,NaN,NaN,NaN
1,4100000000,20,MK,100029,Generator Accessories 650-091 PCB,LKR,0.00,0.00,0.00,3/30/2011,...,004,NaN,NaN,NaN,2011,3,NaN,NaN,NaN,NaN
2,4100000000,30,MK,100029,ENGINE INTERFACE MODULE 630 088  630,LKR,0.00,0.00,0.00,3/30/2011,...,004,NaN,NaN,NaN,2011,3,NaN,NaN,NaN,NaN
3,4100000000,40,MK,100029,Generator Accessories 7.5 kVA Radiator,LKR,0.00,0.00,0.00,3/30/2011,...,004,NaN,NaN,NaN,2011,3,NaN,NaN,NaN,NaN
4,4100000002,10,MK,100157,FCS-AirsysFCB-Inlet/Outlet,USD,0.00,0.00,0.00,4/20/2010,...,001,NaN,NaN,NaN,2010,4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17203,4200001209,10,WK,104724,Removals of equipment related to DTV and,LKR,"5,308,000.00",0.00,"5,308,000.00",8/17/2022,...,B13,Removals of equipment related to DTV and,D00879,NaN,2022,8,REMOVALSOFEQUIPMENTRELATEDTODTVAND,REMOVALSOFEQUIPMENTRELATEDTODTVANDWK20228,NaN,NaN
17204,4200001210,10,WK,102317,"Full Level Repair of LTE, DTV, DE Device",LKR,"114,462,000.00",0.00,"114,462,000.00",4/1/2022,...,T10,"Full Level Repair of LTE, DTV, DE Device",D01783,NaN,2022,4,"FULLLEVELREPAIROFLTE,DTV,DEDEVICE","FULLLEVELREPAIROFLTE,DTV,DEDEVICEWK20224",4.0,65.0
17205,4200001211,10,WK,103209,"Full Level Repair of LTE, DTV, DE Device",LKR,"98,316,000.00",0.00,"98,316,000.00",4/1/2022,...,T10,"Full Level Repair of LTE, DTV, DE Device",D01783,NaN,2022,4,"FULLLEVELREPAIROFLTE,DTV,DEDEVICE","FULLLEVELREPAIROFLTE,DTV,DEDEVICEWK20224",4.0,65.0
17206,4200001212,10,WK,511882,"Full Level Repair of LTE, DTV, DE Device",LKR,"111,617,557.50",0.00,"111,617,557.50",4/1/2022,...,T10,"Full Level Repair of LTE, DTV, DE Device",D01783,NaN,2022,4,"FULLLEVELREPAIROFLTE,DTV,DEDEVICE","FULLLEVELREPAIROFLTE,DTV,DEDEVICEWK20224",4.0,65.0


In [21]:
# Downloading the data set in to local machine
final_data.to_csv('ContractData.csv') 
files.download('ContractData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>